In [2]:
import os.path
import sys
import django
sys.path.append('/home/galm/software/django/tmv/BasicBrowser')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
import scoping
from scoping.models import *

import pandas as pd

p = Project.objects.get(pk=260)
qs = p.query_set.all()
q = Query.objects.get(pk=8843)

dids = set(Doc.objects.filter(query__in=qs).values_list('pk', flat=True))
len(dids)

565084

In [3]:
regenerate_docinfo = False
if regenerate_docinfo:
    df = pd.DataFrame.from_dict(Doc.objects.filter(pk__in=dids).values('id','PY','title','content','wosarticle__dt'))
    df.columns = ["id","Publication Year","Title","Abstract","Document type"]
    df.head()
    df.to_csv('../data/0c_doc_info.csv',index=False)

In [4]:

django.db.connection.close()

dos = DocOwnership.objects.filter(tag__query__project=p, relevant__gt=0)

doc_ids = set(dos.values_list('doc__id',flat=True))

# Get relevant / not relevant decisions
do_df = pd.DataFrame.from_dict(
    dos.values('doc__id','doc__title','doc__content','user__username','relevant')
)
do_df.loc[do_df['relevant']==0,'val'] = np.NaN
do_df.loc[do_df['relevant']==1,'val'] = 1
do_df.loc[do_df['relevant']==2,'val'] = 0
do_df.loc[do_df['relevant']==3,'val'] = np.NaN
do_df.head()

# Get category assignment
cats = Category.objects.filter(project=p).exclude(name__icontains="hidden>")
mitigation = Category.objects.get(project=p, level=2, name__icontains="mitigation")
adaptation_cats = Category.objects.filter(project=p, level=8).exclude(name__icontains="hidden>").exclude(equivalents=mitigation)
cats = cats.exclude(pk__in=adaptation_cats.values_list('pk',flat=True))

df = pd.DataFrame.from_dict(
    DocUserCat.objects.filter(
        category__in=cats,
        doc__in=doc_ids
    ).values(
        'doc__id',
        'doc__title',
        'doc__content',
        'category__name',
        'category__level',
        'user__username',
        #'countries__name',
        'texts__name',
        'selection_tier'
    )
).drop_duplicates(["doc__id","category__name","user__username","texts__name"])
# Filter so that it matches the relevant set
df = df[
    (df[['doc__id','user__username']].apply(tuple, axis=1).isin(do_df[['doc__id','user__username']].apply(tuple, axis=1))) | 
    (df['user__username']=="Auto")
]

# Rename category to capture level
df['category__level'] = df['category__level'].astype(str)
df['Category Name'] = df[['category__level','category__name']].apply(lambda x: ' - '.join(x), axis=1)
#df['cat'] = df[['category__level','category__name','user__username']].apply(lambda x: ' - '.join(x), axis=1)

# Capture the primary/secondary value (1 for primary, 2 for secondary)
df['val'] = df['selection_tier']

# Set the value to the country / text where appropriate and j
#df.loc[~pd.isna(df['countries__name']),'val'] = df.loc[~pd.isna(df['countries__name']),'countries__name']
df.loc[~pd.isna(df['texts__name']),'val'] = df.loc[~pd.isna(df['texts__name']),'texts__name']

df.head()


,doc__id,doc__title,doc__content,category__name,category__level,user__username,texts__name,selection_tier,Category Name,val
0,1859231,PRIORITIZING OCCUPANTS UNDERSTANDING LEVEL ON ...,The building sector is widely known for its gr...,0. Mitigation,2,vasudevan@mcc-berlin.net,None,1,2 - 0. Mitigation,1
1,1868416,Sustainable cements for green buildings constr...,The large amount of waste yearly disposed to l...,0. Mitigation,2,lucy.banisch@icloud.com,None,1,2 - 0. Mitigation,1
2,772501,Design and thermodynamic and thermoeconomic an...,This paper presents the thermodynamic modeling...,0. Not policy related,3,lucy.banisch@icloud.com,None,1,3 - 0. Not policy related,1
3,570203,Comparison of model and observed regional temp...,Results are presented fur six simulations of t...,0. Not policy related,3,lucy.banisch@icloud.com,None,1,3 - 0. Not policy related,1
4,480947,How will auctioning impact on the carbon emiss...,China will enact a national emissions trading ...,countries,11,vasudevan@mcc-berlin.net,None,1,11 - countries,1


In [5]:
# Merge relevant/not relevant with categories
do_df['Category Name'] = "0 - relevant"

merged_df = pd.concat([
    df[['doc__id','doc__title','doc__content','user__username','Category Name','val']],
    do_df[['doc__id','doc__title','doc__content','user__username','Category Name','val']]
])

cats = merged_df['Category Name'].unique()
merged_df.tail()

,doc__id,doc__title,doc__content,user__username,Category Name,val
5630,51374,THE CHOICE BETWEEN AUCTIONING AND GRANDFATHERI...,Based on Hillman's (1982) political support fu...,levs,0 - relevant,1.0
5631,285449,How to support growth with less energy,Economic growth with less use of primary energ...,levs,0 - relevant,1.0
5632,1860974,Global semiconductor industry initiative to re...,Semiconductor manufacture is a small but poten...,levs,0 - relevant,1.0
5633,1299474,Action on climate change requires deliberative...,Despite successful examples of multilevel gove...,levs,0 - relevant,1.0
5634,535895,An Empirical Investigation on the Risk-Return ...,This paper examines the risk-return relationsh...,levs,0 - relevant,1.0


In [25]:
from sqlalchemy import create_engine, inspect, text, URL
from dotenv import dotenv_values

db_config = dotenv_values('.env')
url = URL.create(
    'postgresql',
    db_config['USERNAME'],
    db_config['PW'],
    db_config['HOST'],
    db_config['PORT'],
    db_config['DB']
)
engine = create_engine(url)

with engine.connect() as con:
    result = con.execute(text('''
        SELECT 
            c.name,
            c.code3,
            duc.user_id,
            duc.doc_id
        FROM scoping_docusercat_countries sdc 
        INNER JOIN cities_country c ON sdc.country_id=c.id
        INNER JOIN scoping_docusercat duc ON sdc.docusercat_id=duc.id
        INNER JOIN scoping_category sc on duc.category_id=sc.id
        WHERE sc.project_id=260;
    '''))
    cdf = pd.DataFrame.from_dict(result.all())
    
cdf

,name,code3,user_id,doc_id
0,China,CHN,289,480947
1,Malaysia,MYS,289,1859231
2,China,CHN,289,791223
3,United Kingdom,GBR,289,524618
4,Ghana,GHA,1,643753
...,...,...,...,...
1674,United States,USA,289,236181
1675,Australia,AUS,289,558498
1676,United States,USA,305,643951
1677,Romania,ROU,305,1945936


In [6]:
cats.sort()
cats

array(['0 - relevant', '10 - 3. Quantitative', '10 - 4. Qualitative',
       '11 - countries', '12 - policy name', '2 - 0. Mitigation',
       '2 - 1. Adaptation', '3 - 0. Not policy related',
       '3 - 1. Analysis of an existing policy / concrete proposal',
       '3 - 2. Analysis of a generic policy',
       '3 - 3. Broader governance and policy',
       '4 - 1. Economic instruments', '4 - 2. Regulatory Instruments',
       '4 - 3. Information, education and training',
       '4 - 4. Governance, strategies and targets', '4 - 5. Agreements',
       '5 - 1.02. Carbon pricing', '5 - 1.02. Subsidies',
       '5 - 1.03. Non-carbon taxes',
       '5 - 1.04. Direct Investment / spending', '5 - 2.05. Trade policy',
       '5 - 2.06. Quotas', '5 - 2.07. Regulation on transactions',
       '5 - 2.08. Information requirements',
       '5 - 2.09. Spatial and land-use planning',
       '5 - 2.10. Other regulation', '5 - 2.11. Standards',
       '5 - 3.12. Public Information/awareness campaigns'

In [8]:
# fill tag values with 0s where they have been rated
dudf_wide = (merged_df[['doc__id','doc__title','doc__content','user__username','Category Name','val']]
             .pivot_table(index=['doc__id','doc__title','doc__content','user__username'],columns="Category Name", values="val", aggfunc=np.sum)
             #.fillna(0)
            ).reset_index()

cats = [x for x in cats if x in dudf_wide.columns]

dudf_wide[cats] = dudf_wide[cats].fillna(0)

policy_columns=[x for x in dudf_wide.columns if re.match("^3",x)]
include_policy_columns = [x for x in policy_columns if "not policy related" not in x.lower() and "broader" not in x.lower()]
post_policy_columns=[x for x in dudf_wide.columns if re.match("^[4-9]",x) or re.match("^[0-9]{2}",x)]
#dudf_wide.loc[dudf_wide['3 - 0. Not policy related']==1,post_policy_columns] = np.NaN
dudf_wide.loc[dudf_wide[policy_columns].sum(axis=1,min_count=1)==0,post_policy_columns] = np.NaN

dudf_wide["INCLUDE"] = 0
dudf_wide.loc[(dudf_wide[include_policy_columns]==2).any(axis=1),"INCLUDE"] = 2
dudf_wide.loc[(dudf_wide[include_policy_columns]==1).any(axis=1),"INCLUDE"] = 1
dudf_wide.loc[dudf_wide["2 - 0. Mitigation"]==0, "INCLUDE"] = 0

#dudf_wide.loc[dudf_wide[include_policy_columns].sum(axis=1,min_count=1)==1,"INCLUDE"] = 1
#dudf_wide.loc[dudf_wide[include_policy_columns].sum(axis=1,min_count=1)==0,"INCLUDE"] = 0

print(dudf_wide[dudf_wide['INCLUDE']>0].shape)

dudf_wide.to_csv('../data/0_all_labels.csv',index=False)

dudf_wide.tail()

(1822, 99)


Category Name,doc__id,doc__title,doc__content,user__username,0 - relevant,10 - 3. Quantitative,10 - 4. Qualitative,11 - countries,12 - policy name,2 - 0. Mitigation,...,8 - 02. Buildings,8 - 03. Industry,8 - 04. Energy,8 - 05. Transport,8 - 06. Waste,8 - 15. Cross-sectoral,9 - 0. Ex-post,9 - 1. Ex-ante,9 - 2. Ex-ante (IAM),INCLUDE
5530,3878364,Occupants' Awareness of and Satisfaction with ...,With growing awareness of the need to reduce g...,vasudevan@mcc-berlin.net,1.0,0.0,0.0,0.0,0,1,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
5531,3878440,Urban consumers' willingness to pay for higher...,Rapid urbanization poses accelerating pressure...,ceposadap@gmail.com,1.0,1.0,0.0,1.0,Energy Efficiency Labeling System (EELS),1,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
5532,3878440,Urban consumers' willingness to pay for higher...,Rapid urbanization poses accelerating pressure...,edmondson@hertie-school.org,1.0,0.0,0.0,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5533,3878516,Life cycle greenhouse gas impacts of a connect...,"As technological advancements progress, the au...",ceposadap@gmail.com,1.0,0.0,0.0,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5534,3878516,Life cycle greenhouse gas impacts of a connect...,"As technological advancements progress, the au...",vasudevan@mcc-berlin.net,1.0,0.0,0.0,0.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
